@authors: vcle, bpuhani

All data retrieved in March 2025: <br>
[2020 Population data](https://redistrictingdatahub.org/dataset/oregon-block-pl-94171-2020-by-table/): based on the decennial census at the Census Block level on 2020 Census Redistricting Data

[2020 County data](https://redistrictingdatahub.org/dataset/oregon-county-pl-94171-2020/): from 2020 Census Redistricting Data (P.L. 94-171) Shapefiles

[2020 election data](https://redistrictingdatahub.org/dataset/vest-2020-oregon-precinct-and-election-results/):  VEST 2020 Oregon precinct and election results

[2021 State Senate District plan](https://redistrictingdatahub.org/dataset/2021-oregon-state-senate-adopted-plan/): 2021 Oregon State Senate Approved Plan


<!-- Commented out 2018 Election data for now

[2018 election data](https://redistrictingdatahub.org/dataset/vest-2018-oregon-precinct-and-election-results/)**:**  VEST 2018 Oregon precinct and election results

-->